# 5장 - 클래스와 인터페이스


- 객체지향 언어로서 파이썬은 상속(inheritance), 다형성(polymorphism), 캡슐화(encapsulation)등과 같은 모든 기능을 제공한다.

### BETTER WAY 37 - 내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라

- 파이썬 내장 딕셔너리 타입을 사용하면 객체의 생명 주기 동안 동적인 내부상태를 잘 유지할 수 있다.

In [1]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)
    

book = SimpleGradebook()
book.add_student('뉴턴')
book.report_grade('뉴턴', 90)
book.report_grade('뉴턴', 95)
book.report_grade('뉴턴', 85)
print(book.average_grade('뉴턴'))

90.0


In [4]:
'''
딕셔너리와 관련 내장 타입은 사용하기 너무 쉬우므로 과하게 확장하면서 깨지기 쉬운 코드를 작성할 위험성이 있다.
요구사항이 바뀌어 SimpleGradebook 클래스를 전체 성적이 아니라, 과목별 성적을 리스트로 저장하고 싶다고 하자.
'''
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(score)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count
    
book = BySubjectGradebook()
book.add_student('뉴턴')
book.report_grade('뉴턴', '수학', 75)
book.report_grade('뉴턴', '수학', 65)
book.report_grade('뉴턴', '체육', 90)
book.report_grade('뉴턴', '체육', 95)
print(book.average_grade('뉴턴'))

81.25


In [6]:
'''
요구사항이 또 바뀌어, 이번엔 각 점수의 가중치를 함께 저장해서 중간고사와 기말고사가 더 큰 영향을 미치게 하고 싶다.
'''
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
        
        return score_sum / score_count
    
book = WeightedGradebook()
book.add_student('뉴턴')
book.report_grade('뉴턴', '수학', 75, 0.05)
book.report_grade('뉴턴', '수학', 65, 0.15)
book.report_grade('뉴턴', '수학', 70, 0.80)
book.report_grade('뉴턴', '체육', 100, 0.40)
book.report_grade('뉴턴', '체육', 85, 0.60)
print(book.average_grade('뉴턴'))

80.25


- 하지만, 클래스도 쓰기 어려워졌다. 위치로 인자를 지정하면, 어떤 값이 어떤 뜻을 가지는지 이해하기 어렵다.
- 이와 같은 복잡도가 눈에 들어오면 더 이상 딕셔너리, 튜플, 집합, 리스트 등의 내장 타입을 사용하지 말고, 클래스 계층 구조를 사용해야 한다.

- <font color='blue'>클래스를 활용해 리팩터링하기</font>

- 1)
- collection 내장 모듈에 있는 namedtuple 타입이 이런 경우에 딱 들어맞는다.
- namedtuple 을 사용하면 작은 불변 데이터 클래스를 쉽게 정의할 수 있다.

In [9]:
from collections import namedtuple

# 
Grade = namedtuple('Grade', ('score', 'weight'))

# 일련의 점수를 포함한 단일 과목을 표현하는 클래스 작성
class Subject:
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight
    
# 한 학생이 수강하는 과목들을 표현하는 클래스
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
    
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    
# 마지막으로 모든 학생을 저장하는 컨테이너
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)
        
    def get_student(self, name):
        return self._students[name]

# 다소 길지만 훨씬 더 보기 좋을 것이다.
#
book = Gradebook()
albert = book.get_student('아인슈타인')
math = albert.get_subject('수학')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = albert.get_subject('체육')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(albert.average_grade())

80.25
